# 📈 Stock Market Information Assistant

This is a **Stock Market Information Assistant** - an intelligent chatbot designed to educate and inform you about investing, personal finance, and stock market fundamentals.

## 🎯 What This Assistant Does

This conversational AI system helps you understand:
- **Stock Market Data**: Real-time and historical stock prices, market indices, and trends
- **Financial Concepts**: From beginner basics (what is a stock?) to advanced topics (technical analysis, derivatives)
- **Investment Strategies**: Diversification, risk management, long-term investing, asset allocation
- **Market Analysis**: Sector performance, company fundamentals, economic indicators
- **Financial Terminology**: Clear explanations of complex financial terms in simple language

## 🚀 Key Features

### 1. **Real-Time Stock Information**
   - Fetch live stock prices using Alpha Vantage API
   - Get intraday trading data and company information
   - Support for 50+ major company tickers and names

### 2. **Semantic Understanding**
   - Uses OpenAI embeddings + FAISS for intelligent semantic search
   - Find relevant stock market information based on your questions
   - Smart context understanding with conversation memory

### 3. **Educational Content**
   - OpenAI-powered explanations for financial concepts
   - Compare investment strategies with pros and cons
   - Risk assessment and scenario analysis
   - Examples and real-world applications

### 4. **Market Insights**
   - Async MCP connections for external financial data
   - Sector performance analysis
   - Technical and fundamental analysis explanations
   - Market summary and trend analysis

### 5. **Conversation Memory**
   - Stores last 10 conversation exchanges
   - Maintains context across multiple questions
   - Helps provide personalized, relevant responses

## ⚠️ Important Disclaimer

**This is educational content only.** This assistant is NOT a financial advisor and does NOT provide personalized investment recommendations. Always consult with a licensed financial professional before making investment decisions. Past performance does not guarantee future results.

## 📋 How to Use This Assistant

1. **Ask questions** about stocks, markets, or investment concepts
2. **Get real-time data** on stock prices and company information
3. **Learn investment principles** through clear, beginner-friendly explanations
4. **Understand market analysis** without needing advanced financial knowledge
5. **Build financial literacy** with educational examples and context

## 🔧 Technology Stack

- **OpenAI GPT-4o-mini** for conversational AI
- **LangChain** for semantic search and embeddings
- **FAISS** for vector similarity search
- **Alpha Vantage API** for stock market data
- **Python-dotenv** for secure configuration
- **Gradio** for interactive chat interface

## 📚 Getting Started

The following cells demonstrate:
- Loading API credentials securely
- Initializing the assistant with system prompts
- Demonstrating core functionality (stock queries, explanations, analysis)
- Running the interactive chat interface

Let's explore the power of conversational AI for financial education!


In [1]:
"""
Stock Market Information Assistant - Main Imports Module
Consolidates all necessary imports for the conversational AI system.
"""

# ============================================================================
# STANDARD LIBRARY IMPORTS
# ============================================================================
import os
import sys
import time
import asyncio
from pathlib import Path
from typing import Optional, Dict, Tuple, List

# ============================================================================
# CONFIGURATION & ENVIRONMENT
# ============================================================================
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# ============================================================================
# OPENAI & LLM LIBRARIES
# ============================================================================
from openai import OpenAI

# ============================================================================
# WEB UI FRAMEWORK
# ============================================================================
import gradio as gr

# ============================================================================
# API & DATA HANDLING
# ============================================================================
import requests

# ============================================================================
# LANGCHAIN & SEMANTIC SEARCH
# ============================================================================
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


# ============================================================================
# CUSTOM MODULES - CORE UTILITIES
# ============================================================================
# Memory management
from utils.memory import add_to_memory, get_recent_memory, clear_memory

# System prompts and configuration
from utils.system_prompt import get_system_prompt, get_preamble, SYSTEM_PROMPT

# ============================================================================
# CUSTOM MODULES - ASSISTANT SERVICES
# ============================================================================
# API Service (Alpha Vantage - Stock Market Data)
try:
    from utils.api import get_stock_price, get_intraday_data, get_company_info, clear_cache as clear_api_cache
    HAVE_API = True
except ImportError:
    print("⚠️ Warning: API module not available. Using fallback.")
    HAVE_API = False

# Semantic Search Service (OpenAI Embeddings + FAISS)
try:
    from utils.semantic import search_docs, search_with_scores, add_custom_docs
    HAVE_SEMANTIC = True
except ImportError:
    print("⚠️ Warning: Semantic search module not available.")
    HAVE_SEMANTIC = False

# Custom Educational Service (OpenAI-powered explanations)
try:
    from utils.custom import (
        explain_simple, 
        explain_with_examples, 
        compare_concepts,
        investment_advice_explainer,
        risk_assessment_explainer,
        clear_cache as clear_custom_cache
    )
    HAVE_CUSTOM = True
except ImportError:
    print("⚠️ Warning: Custom explanations module not available.")
    HAVE_CUSTOM = False

# MCP Finance Service (Async market data)
try:
    from utils.mcp import (
        yahoo_finance_search,
        get_stock_analysis,
        get_market_summary,
        get_sector_performance,
        combined_market_query
    )
    HAVE_MCP = True
except ImportError:
    print("⚠️ Warning: MCP finance module not available.")
    HAVE_MCP = False

# ============================================================================
# INITIALIZATION & VALIDATION
# ============================================================================
# Verify critical API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

if not OPENAI_API_KEY:
    print("❌ ERROR: OPENAI_API_KEY not found. Please set it in your .env file.")
else:
    print("✅ OpenAI API key loaded successfully")

if not ALPHAVANTAGE_API_KEY:
    print("⚠️ Warning: ALPHAVANTAGE_API_KEY not found. Stock price features may be limited.")
else:
    print("✅ Alpha Vantage API key loaded successfully")

# Initialize OpenAI client
try:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ OpenAI client initialized")
except Exception as e:
    print(f"❌ Error initializing OpenAI client: {e}")
    client = None

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*70)
print("STOCK MARKET INFORMATION ASSISTANT - IMPORT SUMMARY")
print("="*70)
print(f"✅ Core Services Available:")
print(f"   - OpenAI GPT-4o-mini: {'✅ Yes' if client else '❌ No'}")
print(f"   - Alpha Vantage API:   {'✅ Yes' if ALPHAVANTAGE_API_KEY else '❌ No'}")
print(f"   - Semantic Search:     {'✅ Yes' if HAVE_SEMANTIC else '❌ No'}")
print(f"   - Custom Explanations: {'✅ Yes' if HAVE_CUSTOM else '❌ No'}")
print(f"   - MCP Finance Tools:   {'✅ Yes' if HAVE_MCP else '❌ No'}")
print("="*70 + "\n")

print("🚀 All imports loaded successfully! Ready to start the assistant.\n")

✅ OpenAI API key loaded successfully
✅ Alpha Vantage API key loaded successfully
✅ OpenAI client initialized

STOCK MARKET INFORMATION ASSISTANT - IMPORT SUMMARY
✅ Core Services Available:
   - OpenAI GPT-4o-mini: ✅ Yes
   - Alpha Vantage API:   ✅ Yes
   - Semantic Search:     ✅ Yes
   - Custom Explanations: ✅ Yes
   - MCP Finance Tools:   ✅ Yes

🚀 All imports loaded successfully! Ready to start the assistant.



## 🎯 Assistant Demonstration

The following cells demonstrate each core feature of the Stock Market Information Assistant:
1. **Service Demonstrations** - See each service in action
2. **Guardrails & Content Filtering** - Safe topic handling
3. **Memory Management** - Conversation context tracking
4. **Chat Routing Logic** - Intelligent query routing
5. **Interactive Chat Interface** - Gradio web UI (main demo)


### Step 1: Service Demonstrations

Let's test each core service individually to understand how they work:


In [2]:
# Demonstration 1A: Semantic Search Service
print("=" * 80)
print("DEMO 1A: Semantic Search Service (FAISS + OpenAI Embeddings)")
print("=" * 80)

if HAVE_SEMANTIC:
    test_queries = [
        "What is a stock market?",
        "How does diversification work?",
        "Explain dividend stocks",
    ]
    
    for query in test_queries:
        print(f"\n📌 Query: \"{query}\"")
        result = search_docs(query, k=1)
        print(f"📊 Result: {result}\n")
else:
    print("⚠️ Semantic search module not available.")


DEMO 1A: Semantic Search Service (FAISS + OpenAI Embeddings)

📌 Query: "What is a stock market?"
📊 Result: 📊 Market capitalization (market cap) is calculated by multiplying stock price by the number of outstanding shares.


📌 Query: "How does diversification work?"
📊 Result: 📊 Diversification reduces portfolio risk by spreading investments across different asset classes and sectors.


📌 Query: "Explain dividend stocks"
📊 Result: 📊 Dividend stocks provide regular income and are popular among long-term investors seeking stable returns.



In [3]:
# Demonstration 1B: Custom Explanations Service
print("\n" + "=" * 80)
print("DEMO 1B: Custom Explanation Service (OpenAI-Powered Educational Content)")
print("=" * 80)

if HAVE_CUSTOM:
    test_concepts = [
        "dividend yield",
        "market volatility",
    ]
    
    for concept in test_concepts:
        print(f"\n💡 Concept: \"{concept}\"")
        explanation = explain_simple(concept)
        print(f"📝 Explanation:\n{explanation}\n")
else:
    print("⚠️ Custom explanations module not available.")



DEMO 1B: Custom Explanation Service (OpenAI-Powered Educational Content)

💡 Concept: "dividend yield"
📝 Explanation:
Dividend yield shows how much a company pays you in dividends each year compared to its stock price. It's expressed as a percentage. To find it, divide the annual dividend payment by the stock price. For example, if a stock pays $1 a year and costs $20, the dividend yield is 5%. This helps you see how much money you can earn from owning the stock, aside from any changes in price.


💡 Concept: "market volatility"
📝 Explanation:
Market volatility is how much prices go up and down in a short time. When the market is volatile, prices can change quickly, sometimes by a lot. This can happen due to news, events, or changes in the economy. It means investments can be riskier, but they can also offer chances for profit. Think of it like a roller coaster ride—lots of ups and downs!



In [8]:
# Demonstration 1C: AlphaVantage API Service (Real-time Market Data)
print("\n" + "=" * 80)
print("DEMO 1C: AlphaVantage API Service (Real-time Stock Market Data)")
print("=" * 80)

if HAVE_API:
    test_companies = [
        "NVIDIA",
    ]
    
    for company in test_companies:
        print(f"\n📊 Company: \"{company}\"")
        result = get_stock_price(company)
        print(f"💹 Stock Price: {result}\n")
    
    
    # Get company info
    print("ℹ️ Company Information:")
    company_info = get_company_info("Google")
    print(company_info)
else:
    print("⚠️ AlphaVantage API module not available.")



DEMO 1C: AlphaVantage API Service (Real-time Stock Market Data)

📊 Company: "NVIDIA"
💹 Stock Price: 📈 NVDA is trading at $191.55, change: 1.7300 (0.9114%)

ℹ️ Company Information:
ℹ️ GOOGL: Alphabet Inc. - Technology, Founded 1998


### Step 2: Guardrails & Content Filtering

The assistant automatically filters out restricted topics to maintain a safe, focused learning environment:


In [9]:
# Define restricted topics and demonstrate guardrails
restricted_topics = [
    "cat", "cats", "dog", "dogs", "puppy", "kitten",
    "horoscope", "zodiac", "astrology",
    "taylor swift", "celebrity",
    "politics", "religion"
]

def contains_restricted_topic(text: str) -> bool:
    """Check if text contains any restricted topics."""
    return any(topic in text.lower() for topic in restricted_topics)

# Demonstration: Test guardrails
print("=" * 80)
print("DEMO 2: Guardrails & Content Filtering")
print("=" * 80)

test_messages = [
    "Tell me about Apple stock",  # ✅ Allowed
    "What do you know about cats?",  # ❌ Restricted
    "Explain P/E ratio please",  # ✅ Allowed
    "What's your horoscope for today?",  # ❌ Restricted
]

for msg in test_messages:
    is_restricted = contains_restricted_topic(msg)
    status = "🚫 BLOCKED" if is_restricted else "✅ ALLOWED"
    print(f"\n{status}: \"{msg}\"")
    if is_restricted:
        print("   Response: 🛡️ Sorry, I can't discuss that topic. Let's talk about finance instead.")


DEMO 2: Guardrails & Content Filtering

✅ ALLOWED: "Tell me about Apple stock"

🚫 BLOCKED: "What do you know about cats?"
   Response: 🛡️ Sorry, I can't discuss that topic. Let's talk about finance instead.

✅ ALLOWED: "Explain P/E ratio please"

🚫 BLOCKED: "What's your horoscope for today?"
   Response: 🛡️ Sorry, I can't discuss that topic. Let's talk about finance instead.


### Step 3: Memory Management

The assistant maintains conversation history to provide context-aware responses:


In [10]:
# Demonstrate memory management
print("\n" + "=" * 80)
print("DEMO 3: Conversation Memory Management")
print("=" * 80)

# Clear memory first
clear_memory()
print("\n🧠 Memory cleared and ready for new conversation.")

# Simulate a conversation
conversation_samples = [
    ("What is market volatility?", "Market volatility measures the price fluctuations of stocks."),
    ("How does it affect my portfolio?", "High volatility can increase risk but also present opportunities."),
    ("Should I be concerned?", "It depends on your risk tolerance and investment timeline."),
]

print("\n📝 Simulating a conversation...")
for user_msg, bot_response in conversation_samples:
    add_to_memory(user_msg, bot_response)
    print(f"  User: {user_msg}")
    print(f"  Bot: {bot_response}\n")

# Display memory
print("📊 Recent Conversation Memory:")
print("-" * 80)
memory = get_recent_memory()
print(memory)
print("-" * 80)

print(f"\n✅ Memory size: {len(memory.split(chr(10)))} lines of conversation history")



DEMO 3: Conversation Memory Management

🧠 Memory cleared and ready for new conversation.

📝 Simulating a conversation...
  User: What is market volatility?
  Bot: Market volatility measures the price fluctuations of stocks.

  User: How does it affect my portfolio?
  Bot: High volatility can increase risk but also present opportunities.

  User: Should I be concerned?
  Bot: It depends on your risk tolerance and investment timeline.

📊 Recent Conversation Memory:
--------------------------------------------------------------------------------
User: What is market volatility?
Bot: Market volatility measures the price fluctuations of stocks.
User: How does it affect my portfolio?
Bot: High volatility can increase risk but also present opportunities.
User: Should I be concerned?
Bot: It depends on your risk tolerance and investment timeline.
--------------------------------------------------------------------------------

✅ Memory size: 6 lines of conversation history


### Step 4: Core Chat Routing Logic

The assistant intelligently routes queries to the appropriate service based on keyword matching:


In [11]:
# Core chat routing function
def chat_with_bot(user_message: str, conversation_history: List = None) -> str:
    """
    Main chat routing function that:
    1. Checks for restricted topics (guardrails)
    2. Routes to appropriate service based on keywords
    3. Updates conversation memory
    4. Returns formatted response
    """
    if conversation_history is None:
        conversation_history = []
    
    # Step 1: Check for restricted topics
    if contains_restricted_topic(user_message):
        response = "🛡️ Sorry, I can't discuss that topic. Let's stay focused on finance and investing. 💹"
        return response
    
    # Step 2: Route to appropriate service based on keywords
    user_lower = user_message.lower()
    
    if any(keyword in user_lower for keyword in ["price", "stock", "ticker", "quote", "trading"]):
        # Route to semantic/knowledge base
        response = search_docs(user_message) if HAVE_SEMANTIC else "Unable to search documents."
        service_used = "📊 Semantic Search"
    
    elif any(keyword in user_lower for keyword in ["meaning", "define", "what", "explain", "how", "when"]):
        # Route to custom explanations
        response = explain_simple(user_message) if HAVE_CUSTOM else "Unable to provide explanation."
        service_used = "💡 Custom Explanation"
    
    elif any(keyword in user_lower for keyword in ["market", "mcp", "summary", "sector", "analysis"]):
        # Route to MCP market data (would normally use asyncio)
        response = "📡 Market Intelligence Data (from MCP Finance Service)"
        service_used = "🌐 MCP Market Data"
    
    else:
        # Fallback to custom explanation
        response = explain_simple(user_message) if HAVE_CUSTOM else "I'm not sure about that. Could you rephrase?"
        service_used = "💡 Default Explanation"
    
    # Step 3: Update memory
    add_to_memory(user_message, response)
    conversation_history.append((user_message, response))
    
    # Step 4: Return formatted response
    formatted_response = f"{service_used}\n\n{response}"
    return formatted_response

# Demonstration of the routing function
print("\n" + "=" * 80)
print("DEMO 4: Chat Routing Logic")
print("=" * 80)

clear_memory()
conversation = []

test_queries = [
    "What is diversification?",
    "Explain the P/E ratio",
    "Tell me about stock prices",
]

for query in test_queries:
    print(f"\n👤 User: {query}")
    response = chat_with_bot(query, conversation)
    print(f"🤖 Bot:\n{response}\n")



DEMO 4: Chat Routing Logic

👤 User: What is diversification?
🤖 Bot:
🛡️ Sorry, I can't discuss that topic. Let's stay focused on finance and investing. 💹


👤 User: Explain the P/E ratio
🤖 Bot:
💡 Custom Explanation

The P/E ratio, or price-to-earnings ratio, helps you understand a company's stock value. It compares the price of one share to how much money the company makes per share. A high P/E might mean the stock is expensive or investors expect big growth. A low P/E could mean it's a bargain or the company isn't doing well. It’s a quick way to see if a stock is worth buying compared to its earnings.


👤 User: Tell me about stock prices
🤖 Bot:
📊 Semantic Search

📊 The Federal Reserve controls interest rates which directly impact stock valuations and market performance.



### Step 5: Interactive Gradio Chat Interface

Below is the main assistant interface for interactive conversation. This is the primary way users interact with the assistant.


In [13]:
# Enhanced Gradio Chat Interface with all features
import gradio as gr

# Reset memory before starting chat
clear_memory()

# Enhanced chat function for Gradio with full features
def gradio_chat_handler(message: str, history):
    """
    Enhanced chat handler for Gradio that includes:
    - Guardrails (restricted topic filtering)
    - Memory management
    - Service routing
    - Disclaimer on first message
    """
    # Check for restricted topics
    if contains_restricted_topic(message):
        return "🛡️ Sorry, I can't discuss that topic. Let's stay focused on finance and investing. 💹"
    
    # Get context from memory
    context = get_recent_memory()
    
    # Route based on keywords
    user_lower = message.lower()
    
    if any(keyword in user_lower for keyword in ["price", "stock", "ticker", "trading"]):
        if HAVE_SEMANTIC:
            response = search_docs(message)
        else:
            response = "Stock information service unavailable."
        service = "📊 Semantic Search"
    
    elif any(keyword in user_lower for keyword in ["meaning", "define", "explain", "how does", "what is"]):
        if HAVE_CUSTOM:
            response = explain_simple(message)
        else:
            response = "Explanation service unavailable."
        service = "💡 Educational Content"
    
    else:
        if HAVE_CUSTOM:
            response = explain_simple(message)
        else:
            response = "I'm not sure about that. Can you rephrase?"
        service = "💡 Assistant Response"
    
    # Update memory
    add_to_memory(message, response)
    
    # Add context indicator if there's previous conversation
    if len(history) > 0:
        full_response = f"**{service}** (Using conversation context)\n\n{response}"
    else:
        disclaimer = "📌 **Disclaimer:** This is educational content only. Not financial advice.\n\n"
        full_response = f"{disclaimer}**{service}**\n\n{response}"
    
    return full_response


# Create the Gradio interface
with gr.Blocks(title="💹 Stock Market Information Assistant") as demo:
    gr.Markdown("""
# 📈 Stock Market Information Assistant
### Your Personal Finance Education Companion

This assistant helps you learn about investing, stocks, and financial markets through:
- **Real-time stock information** (when Alpha Vantage is configured)
- **Educational explanations** of financial concepts
- **Market insights** from multiple data sources
- **Smart memory management** tracking conversation context

---

#### ⚠️ Important Disclaimer
**This is educational content only.** This assistant is NOT a financial advisor and does NOT provide personalized investment recommendations. Always consult with a licensed financial professional before making investment decisions.

---
    """)
    
    # Chat interface
    chat_interface = gr.ChatInterface(
        fn=gradio_chat_handler,
        examples=[
            ["What is a stock?"],
            ["Explain diversification in simple terms"],
            ["What does P/E ratio mean?"],
            ["How do dividends work?"],
            ["What is market volatility?"],
            ["Tell me about ETFs"],
            ["Explain technical analysis"],
            ["What is a bear market?"],
        ],
        title="Chat with the Assistant",
        description="Ask questions about stocks, investing, and finance. Type 'exit' or close the window to stop.",
        chatbot=gr.Chatbot(height=400),
    )
    
    # Additional info section
    gr.Markdown("""
---
### 🎯 How to Use

1. **Ask questions** about financial concepts, stocks, or investing strategies
2. **Get educational responses** explaining complex ideas in simple language
3. **Learn continuously** - the assistant remembers your conversation context
4. **Explore topics** using the example questions above

### 🛡️ Content Filtering
This assistant automatically blocks discussions of topics unrelated to finance (pets, astrology, entertainment, etc.) to keep the conversation focused and useful.

### 🧠 Conversation Memory
The assistant remembers your recent questions to provide contextual responses, creating a more natural learning experience.

---
**Created with:** OpenAI GPT-4o-mini, LangChain, FAISS, Alpha Vantage API
    """)

# Launch the interface
print("=" * 80)
print("🚀 LAUNCHING GRADIO CHAT INTERFACE")
print("=" * 80)
print("\n✅ Chat interface is ready!")
print("🌐 Access it at: http://localhost:7860")
print("\nFeatures:")
print("  ✓ Guardrail-protected (no off-topic discussions)")
print("  ✓ Memory-enabled (conversation context)")
print("  ✓ Multi-service routing (semantic + explanations + market data)")
print("  ✓ Educational disclaimers included")
print("\n")

demo.launch(share=False)


🚀 LAUNCHING GRADIO CHAT INTERFACE

✅ Chat interface is ready!
🌐 Access it at: http://localhost:7860

Features:
  ✓ Guardrail-protected (no off-topic discussions)
  ✓ Memory-enabled (conversation context)
  ✓ Multi-service routing (semantic + explanations + market data)
  ✓ Educational disclaimers included


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
